3. Selenium 심화 : 브라우저 제어
    - 웹 테스트 자동화 도구인 Selenium을 이용해
      브라우저를 제어하는 방법을 배웁니다.

목 차

01. Selenium을 이용한 브라우저 제어

02. 브라우저 로딩 기다리기

03. 키보드/마우스 입력

04. 다양한 입력, ActionChains

05. 맺으며

01. Selenium을 이용한 브라우저 제어

로그인이 필요한 페이지

driver.get("https://mail.naver.com")
e_list = driver.find_elements_by_xpath('//div/p')

for e in e_list:
    print(e.text)
    
실행 결과
-> 로그인 필요

동적으로 렌더링되는 페이지

드랍다운 내리면 html이 있지만
닫으면 사라짐

브라우저 제어

1) 로그인 자동화
    - 로그인 후에만 나오는 웹페이지 분석을 위해
    - ID와 비밀번호 입력 후 로그인 버튼 클릭(또는 엔터키)
    => 어떻게?
    
2) 드랍다운 버튼 클릭
    - 드랍다운 버튼을 클릭해야만 나오는 요소의 추출을 위해
    - 드랍다운 버튼을 찾아서 클릭
    => 어떻게?
    
답은 Selenium이다!

02. 브라우저 로딩 기다리기

Scraping 동작 순서

Python
with webdriver.Firefox() as driver:
    driver.get(url) # 웹 페이지 로딩
    
    e = driver.find_element()
    ....
    
    
< 이상적인 경우 >
with webdriver.Firefox() as driver:
             ↓
      driver.get(url) ----------↓
             ↓                  ↓
             ↓        ←   해당 웹 페이지 로딩 
             ↓
      e = driver.find_element()
             ↓
           .....

Scraping 동작 순서 : 지연된 로딩

Python
with webdriver.Firefox() as driver:
    driver.get(url) # 웹 페이지 로딩
    
    e = driver.find_element() # error !
    ....
    
    
    
< 네트워크 문제 등으로 로딩이 늦을 경우 >
with webdriver.Firefox() as driver:
             ↓
      driver.get(url) -----------↓
             ↓                  ↓
             ↓                  ↓
             ↓                  ↓
      e = driver.find_element()  ↓ # Error
             ↓     ← ← ← ←   해당 웹 페이지 로딩 
           .....

기다리기 : 3가지

무조건 기다리기


Python
import time

with webdriver.Firefox() as driver:
    driver.get(url) # 웹페이지 로딩
    time.sleep(10) # 10초간 기다리기
    
    e = driver.find_element()
    ....
    
time.sleep()
- 파이썬 내장 라이브러리
- n초만큼 무조건 기달림
- 웹페이지 로딩이 끝났어도 주어진 시간을
  계속 기다리기 때문에 비효율적

암시적 기다리기

Python
with webdriver.Firefox() as driver:
    driver.implicitly_wait(10) # 10초
    
    driver.get(url) # 웹페이지로딩
    
    e = driver.find_element()
    ....

implicitly_wait(n)
- 암시적으로 기다림 수행
- 웹 페이지를 새로 로딩할 때마다 (driver.get)
  최대 n초까지 기다림 
- 웹 페이지 로딩이 끝나면 기다리기를 종료하고 코드를 수행 v
- 한 번만설정해주면 계속해서 적용

특정 요소만 로딩이 지연되는 경우

웹페이지는 로딩이 됐으나, 요소가 불러와지지 않은 경우

지연된 로딩 예시

페이지 로딩이 끝남 -> 잠시 후에 광고가 보인다.

광고 JS를 통해 넣게 되는데
Error 발생

명시적 기다리기

Python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

with webdriver.Firefox() as driver:
    driver.implicitly_wait(10) # 10초
    driver.get(url) # 웹페이지로딩
    e = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.ID,"id_name"))
       ) # 해당 요소를 불러올 때까지 10초 기다림
       
명시적 기다리기
- 기다릴 요소를 명시
- 명시된 요소가 해당 방식으로 불러와
  질 때까지 최대 n초 기다림
- 요소는 class_name, xpath등 다양한 방법으로 찾을 수 있음

presence_of_element_located
- 괄호 안의 요소가 나타날 때까지 기다림

명시적 기다리기

Python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

with webdriver.Firefox() as driver:
    driver.implicitly_wait(10) # 10초
    driver.get(url) # 웹페이지로딩
    e = WebDriverWait(driver, 10).until(
       EC.element_to_be_clickable((By.ID,"id_name"))
       ) # 해당 요소가 클릭 가능해질때까지 10초 기다림
       
(By.ID, "id_name")
- 요소를 찾는 방법과 파라미터를 의미
- id 외에 대표적으로 사용하는 것
    - (By.XPATH, "xpath")
    - (By.CLASS, "class_name")

element_to_be_clickable
- 해당 요소가 클릭 가능해질 때까지 기다림

웹 페이지 로딩 기다리기
이번에 실습할 웹 페이지는, 처음에는 아래와 같이 텍스트만 덩그러니 있습니다.

image

하지만 약 3초 뒤에 이미지가 불러와집니다.

image

이미지가 3초 뒤에 불러와지는 웹 페이지를 이용해서 wait 를 학습해봅시다.


지시사항
먼저 wait 없이 브라우저의 로딩이 끝난 직후 요소를 추출하고, src 속성값을 출력합니다.

다시 웹 페이지에 접속하고, 이번엔 time.sleep() 을 이용하여 10초간 로딩을 기다렸다 요소를 추출하고, src 속성값을 출력합니다.

다시 웹 페이지에 접속하고, 이번엔 implicitly_wait() 을 이용하여 최대 10초간 로딩을 기다렸다 요소를 추출하고, src 속성값을 출력합니다.

다시 웹 페이지에 접속하고, 이번엔 요소를 명시하여 최대 10초간 로딩을 기다렸다 요소를 추출하고, src 속성값을 출력합니다.

Tips!
요소의 속성값을 출력하기 위해선 아래와 같이 작성합니다.
e.get_attribute("src")
Copy
요소를 명시하여 기다릴 때에는 다음과 같이 작성하면 됩니다.
e = WebDriverWait(driver, n).until(EC.presence_of_element_located((By.TAG_NAME, "img")))
Copy

In [ ]:
# index.html
'''
<!DOCTYPE html>
<html>
  <head>
    <script src="vue.min.js"></script>
    <meta charset="UTF-8" />
    <title>Selenium 실습</title>
  </head>

  <body>
    <h3>엘리스 로고</h3>

    <script>
      const vueApp = new Vue({
        el: "#app",

        data() {
          return {};
        },

        created() {
          setTimeout(() => {
            const img = document.createElement("img");

            img.setAttribute("src", "elice_logo.png");
            img.setAttribute("alt", "엘리스 로고");

            document.body.appendChild(img);
          }, 3000);
        },

        async mounted() {},

        methods: {
          async sleep(ms) {
            const wakeUpTime = Date.now() + ms;
            while (Date.now() < wakeUpTime) {}
          },
        },

        watch: {},
      });
    </script>
  </body>
</html>

'''

In [ ]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

with webdriver.Firefox() as driver:
    driver.get("http://localhost:8080")

    # 지시사항 1번을 작성하세요.
        # 먼저 wait 없이 브라우저의 로딩이 끝난 직후 요소를 추출하고,
        # src 속성값을 출력합니다.
    try:
        img = driver.find_element_by_tag_name('img')
        print(img.get_attribute('src'))
    except Exception:
        print("아직 요소가 불러와지지 않았습니다.")

    driver.get("http://localhost:8080")
    # 지시사항 2번을 작성하세요.
        # 다시 웹 페이지에 접속하고, 이번엔 time.sleep()을 이용하여
        # 10초간 로딩을 기다렸다가 요소를 추출하고, src 속성값을 출력합니다.
    try:
        time.sleep(10)
        img = driver.find_element_by_tag_name('img')
        print(img.get_attribute('src'))
    except Exception:
        print("아직 요소가 불러와지지 않았습니다.")

    driver.get("http://localhost:8080")
    # 지시사항 3번을 작성하세요.
    
        # 다시 웹 페이지에 접속하고, 
        # 이번엔 암시적으로 implicitly_wait() 을 이용하여
        # 최대 10초간 로딩을 기다렸다 요소를 추출하고, src 속성값을 출력합니다.
    driver.implicitly_wait(10)
    try:
        img = driver.find_element_by_tag_name('img')
        print(img.get_attribute('src'))

    except Exception:
        print("아직 요소가 불러와지지 않았습니다.")


with webdriver.Firefox() as driver:
    driver.get("http://localhost:8080")
    # 지시사항 4번을 작성하세요.
        # 다시 웹 페이지에 접속하고, 
        # 이번엔 요소를 명시하여 최대 10초간 로딩을 기다렸다가
        # 요소를 추출하고, src 속성값을 출력합니다.
    try:
        img = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "img")))
        print(img.get_attribute('src'))
    except Exception:
        print("아직 요소가 불러와지지 않았습니다.")


03. 키보드/마우스 입력

로그인을 해야 볼 수 있는 페이지

로그인이 필요한 서비스는 어김없이 해당 페이지가 뜬다.

키보드/마우스 입력으로 로그인 자동화

참고

XPath 
<input id="id"...># 아이디 입력창
<input id="pw"...># 패스워드 입력창
<input id="log.login"...> # 버튼

키보드 입력

Python
from selenium.webdriver.common.keys import Keys

#id 입력
driver.find_element_by_xpath('//[@id="id"]'\
.send_keys('my_id')

#password 입력 + 엔터
driver.find_element_by_xpath('//*[@id="pw"]')\
.send_keys('my_password' + Keys.ENTER)


send_keys
- 해당 요소에 나열된 key sequence를 순차적으로 입력
- 예시에선 m,y,_,i,d를 순차적으로 입력

Keys
- 특수 키를 쓸수 있게 해주는 class
- ENTER, CONTROL,SHIFT등 모든 특수 키가 있으며,
  항상 대분자로 작성

마우스 입력

Python
#id 입력
driver.find_element_by_xpath('//*[@id="id"]')\
.send_keys('my_id')

#pw입력 + 엔터
driver.find_element_by_xpath('//*[@id=""pw]')\
.send_key('my_pw' + Keys.ENTER)

#로그인 버튼 클릭
driver.find_element_by_xpath('//*[@id="log.login"]')\
.click()


click
- 해당 요소를 클릭하는 효과

키보드/마우스 입력
다음은 로그인 테스트 페이지입니다.

image

ID와 비밀번호를 콘솔 에 입력하면, Selenium의 기능을 이용해 창에 자동으로 입력되고, 로그인 버튼을 클릭해서 로그인이 되도록 해봅시다.

키보드 입력 과 마우스 클릭 은 각각

e.send_keys("my_id")
Copy
e.click()
Copy
을 이용합니다.


지시사항
ID 를 입력할 요소를 찾습니다. 2장에서 배웠던 내용을 토대로 자유롭게 요소를 찾아봅시다.

위에서 찾은 요소에 send_keys() 메써드를 이용하여, 콘솔 창에 입력했던 ID 정보가 담긴 input_id 를 로그인 창에 입력합니다.

비밀번호를 입력할 요소를 찾습니다. 마찬가지로 send_keys() 메써드를 이용하여 input_pw 를 로그인 창에 입력합니다.

로그인 버튼 요소를 찾습니다. click() 메써드를 이용하여 로그인을 합니다.

로그인 시도 후 뜨는 메시지가 담긴 요소를 찾고, 그 요소의 text를 출력합니다.

Tips!
회원가입같은 절차 없이, 조건에 맞게 ID와 비밀번호를 입력하면 로그인이 됩니다.
ID: 8자 이상의 문자열 (영어, 숫자 허용)
비밀번호: 영어, 숫자가 혼합된 8자 이상의 문자열

In [ ]:
from selenium import webdriver

with webdriver.Firefox() as driver:
    driver.get("http://localhost:8080")

    input_id = input()
    input_pw = input()

    # 지시사항 1번과 2번을 작성하세요.
        # 1. ID를 입력할 요소를 찾습니다.
        #    2장에서 배웠던 내용을 토대로 자유롭게 요소를 찾아봅시다.
    id_element = driver.find_element_by_id('id')
    
        # 2. 위에서 찾은 요소에 send_keys() 메소드를 이용하여
        #    콘솔 창에 입력했던 ID 정보가 담긴 input_id를 로그인창에
        #    입력합니다.
    id_element.send_keys(input_id)
    
    
    # 지시사항 3번을 작성하세요.
        # 3. 비밀번호를 입력할 요소를 찾습니다.
        #    마찬가지로 send_key=() 메소드를 이용하여
        #    input_pw를 로그인창에 입력합니다.
    pw_element = driver.find_element_by_id('pw')
    pw_element.send_keys(input_pw)
    # 지시사항 4번을 작성하세요.
        # 4. 로그인 버튼 요소를 찾습니다.
        #    click() 메서드를 이용하여 로그인을 합니다.
    button = driver.find_element_by_id('login')
    button.click()
    # 지시사항 5번을 작성하세요.
        # 5. 고르인 시도 후 뜨는 메시지가 담긴 요소를 찾고,
        #    그 요소의 text를 출력합니다.
    message = driver.find_element_by_id('message')
    print(message.text)

04. 다양한 입력, ActionChains

기본 입력의 한계

Python
#id 입력
driver.find_element_by_xpath('//*[@id="id"]')\
.send_keys('my_id')

#pw입력 + 엔터
driver.find_element_by_xpath('//*[@id=""pw]')\
.send_key('my_pw' + Keys.ENTER)

#로그인 버튼 클릭
driver.find_element_by_xpath('//*[@id="log.login"]')\
.click()


각 요소에 key sequence를 입력하거나,
마우스 클릭 등의 액션을 할 수 있지만,
더 이상의 복잡한 액션은 할 수 없음

ActionChains

여러가지의 action을 chain처럼 엮어 수행하는 기능
이를 통해 좀 더 복잡한 액션을 할 수 있음
(더블 클릭, ctrl + 클릭 등)

ActionChains 사용법

Python
webdriver.ActionChains(driver)\
    .action1()\
    .action2()\
    .....
    .perform() # 1st action chains
webdriver.ActionChains(driver)\
    .action1()\
    .action2()\
    .....
    .perform() # 2nd action chains
    
    
Python #인스턴스
chains = webdriver.ActionChains(driver)
chains.action1()
chains.action2()
...
chains.perform() # 1st action chains

chains.action1()
chains.action2()
...
chains.perform() # 2nd action chains

ActionChains의 키보드 입력

Python
#요소 찾기
_id = driver.find_element_by_xpath('//*[@id="id"]')
_pw = driver.find_element_by_xpath('//*[@id="pw"]')

#액션 수행
webdriver.ActionChains(driver)\
    .send_keys_to_element(_id, 'my_id')\
    .send_keys_to_element(_pw, 'my_pw')\
    .send_keys(Keys.ENTER)\
    .perform()
    
send_keys_to_element
- 주어진 요소에 주어진 key sequence를 입력
- 하나의 ActionChains 내에서 여러 오소에 액션을 수행할 수 있음

send_keys
- 요소가 주어지지 않기 때문에,
  현재 커서가 위치한 곳에서 key sequence를 입력
- 예시에선 _pw(비밀번호 입력 칸)에서 ENTER를 입력

ActionChains의 다양한 method 조합

Python
#컨트롤 + click (우클릭과 같은 효과)
_button = driver.find_element_by_xpath('//button')
chains = webdriver.ActionChains(driver)

chains.key_down(Keys.CONTROL) # 컨트롤 버튼 누른 채로
chains.click(_button) # 버튼 요소 클릭
chains.perform()

ActionChains의 다양한 method 조합

Python
#마우스 커서를 올렸다, 3초 뒤에 떼기
_button = driver.find_element_by_xpath('//button')
chains = webdriver.ActionChains(driver)

chains.move_to_element(_button)\
    .pause(3)\
    .move_by_offset(100, 100)\
    .perform()
    
move_to_element
- 주어진 요소로 마우스 커서를 이동

pause
- n초만큼 멈췄다 뒤의 액션을 수행

move_by_offset
- x,y 축으로 주어진 수치만큼 이동

ActionChains의 활용
저번 실습에서 했던 로그인 자동화를, ActionChains를 이용하여 다시 해보는 시간을 가집니다.

image

ID와 비밀번호를 콘솔 에 입력하면, Selenium의 기능을 이용해 창에 자동으로 입력되고, 로그인 버튼을 클릭해서 로그인이 되도록 해봅시다.

키보드 입력 과 마우스 클릭 은 각각

chains.send_keys_to_element(e, "my_id")
Copy
chains.click(e)
Copy
을 이용합니다.


지시사항
ID, 비밀번호, 로그인 버튼에 해당하는 요소를 찾아 각각 id_e, pw_e, login_e 에 넣습니다.

ActionChains의 인스턴스를 만들고, chains 에 넣습니다.

chains.send_keys_to_element() 를 이용해서 ID와 비밀번호를 입력합니다. id_e 요소에는 input_id를, pw_e 요소에는 input_pw를 입력하면 됩니다.

chains.click() 을 이용해서 login_e요소를 클릭합니다.

chains.perform() 을 작성하여 지금까지 ActionChains에 저장된 모든 액션을 순서대로 수행합니다.

로그인 시도 후 뜨는 메시지가 담긴 요소를 찾고, 그 요소의 text를 출력합니다.

Tips!
ActionChains 를 쓰는 방법은 두 가지가 있습니다.
Class Method 로 사용
webdriver.ActionChains(driver).send_keys_to_element(e).click().perform()
Copy
인스턴스로 만들어 사용
chains = webdriver.ActionChains(driver)
chains.send_keys_to_element(e)
chains.click()
chains.perform()

In [ ]:
from selenium import webdriver

with webdriver.Firefox() as driver:
    driver.get("http://localhost:8080")

    input_id = input()
    input_pw = input()

    # 지시사항 1번을 작성하세요.
    # ID, 비밀번호, 로그인 버튼에 해당하는 요소를 찾아 
    # 각각 id_e, pw_e, login_e 에 넣습니다.
    id_e = driver.find_element_by_id('id')
    pw_e = driver.find_element_by_id('pw')
    login_e = driver.find_element_by_id('login')
    # 지시사항 2번을 작성하세요.
    # ActionChains의 인스턴스를 만들고, chains 에 넣습니다.
    chains = webdriver.ActionChains(driver)

    # 지시사항 3번을 작성하세요.
    # chains.send_keys_to_element() 를 이용해서 ID와 비밀번호를 입력합니다. 
    # id_e 요소에는 input_id를, pw_e 요소에는 input_pw를 입력하면 됩니다.
    chains.send_keys_to_element(id_e,input_id)
    chains.send_keys_to_element(pw_e,input_pw)
    # 지시사항 4번을 작성하세요.
    # chains.click() 을 이용해서 login_e요소를 클릭합니다.
    chains.click(login_e)
    # 지시사항 5번을 작성하세요.
    # chains.perform() 을 작성하여 
    # 지금까지 ActionChains에 저장된 모든 액션을 순서대로 수행합니다.
    chains.perform()
    
    message = driver.find_element_by_id('message')
    print(message.text)

05. 맺으며

웹 페이지 로딩을 기다리는 방법 3가지

1) 무조건 기다리기
    - time.sleep(n) / python 내장 라이브러리
    - n초만큼 무조건 기다림
    - 웹 페이지 로딩이 끝났어도 주어진 시간을 계속 
      기다리기 때문에 비효율적
      
2) 암시적 기다리기
    - driver.implicitly_wait(n)
    - 웹 페이지를 새로 로딩하게 되면 매번 n초까지 기달림
    - 웹 페이지의 로딩이 끝나면 기다리기를 종료하고 코드를 수행
    
3) 명시적 기다리기
    - 기다릴 요소를 명시
    - 명시된 요소가 해당 방식으로 불러와질 때까지 최대 n초 기다림
    - 요소는 class_name.xpath등 다양한 방법으로 찾을 수 있음

키보드와 마우스를 입력

입력 기본
#id 입력
driver.find_element_by_xpath('//*[@id="id"]')\
    .send_keys("my_id")
    
#pw 입력 + 엔터
driver.find_element_by_xpath('//*[@id="pw"]')\
    .send_keys("my_pw" + Keys.ENTER)
    
#로그인 버튼 클릭
driver.find_element_by_xpath('//*[@id="log.login"]')\
    .click()
    


ActionChains
_id = driver.find_element_by_xpath('//*[@id="id"]')
_pw = driver.find_element_by_xpath('//*[@id="pw"]')

#액션 수행
webdriver.ActionChains(driver)\
    .send_keys_to_element(_id,'my_id')\
    .send_keys_to_element(_pw,'my_pw')\
    .send_keys(Keys.ENTER)\
    .perform()

Selenium을 이용한 브라우저 제어
드디어 3장 강의가 완료되었군요! 수고하셨습니다~
파이썬 크롤링의 마지막 장으로 넘어가기 전에 남은 것은..? 🤔
바로 복습이죠ㅎㅎ 지금 시작합니다 🏁



1. 브라우저 제어
Q. 브라우저 제어는 왜 해야할까요?

로그인 자동화

로그인 후에만 나오는 웹 페이지 분석을 위해
ID와 비밀번호 입력 후 로그인 버튼 클릭 (또는 엔터키 입력)
드랍다운 버튼 클릭

드랍다운 버튼을 클릭해야만 나오는 요소의 추출을 위해
드랍다운 버튼을 찾아서 클릭
→ 답은 Selenium입니다!!



2. 브라우저 로딩 기다리기
브라우저 로딩이 지연되는 경우가 있습니다.

이럴 때는 어떻게 해야할까요?

Point I
time.sleep(n): 무조건 기다리기

import time

with webdriver.Firefox() as driver:
    driver.get(url) #  웹페이지 로딩
    time.sleep(10) # 10초간 기다리기
    
    e = driver.find_element()
    ...
python 내장 라이브러리
n초만큼 무조건 기다림
내가 원하는 요소가 불러와 졌어도 주어진 시간을 계속 기다림
Point II
implicitly_wait(n): 암시적 기다리기

with webdriver.Firefox() as driver:
    driver.implicitly_wait(10)
    driver.get(url) # 웹 페이지 로딩
    
    e = driver.find_element()
    ...
암시적으로 기다림 수행
앞으로 요소를 추출할 때 (find_element) 최대 n초까지 기다림
해당 요소의 로딩이 끝나면 즉시 기다리기를 종료하고 코드를 수행
한 번 설정해주면 해당 브라우저에서 계속해서 적용
Point III
presence_of_element_located: 명시적 기다리기

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

with webdriver.Firefox() as driver:
    driver.implicitly_wait(10)
    driver.get(url) # 웹 페이지 로딩
    
    e = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.ID, "id_name"))
        # 괄호 안의 요소가 나타날 때까지 기다림
    ) # 해당 요소를 불러올 때까지 30초 기다림
    ...
명시된 요소가 해당 방식으로 불러와질 때까지 최대 n초 기다림
요소는 class_name, xpath 등 다양한 방법으로 찾을 수 있음
다른 예시)

element_to_be_clickable: 해당 요소가 클릭 가능해질 때까지 기다림


3. 키보드/마우스 입력
Point I
키보드 입력

from selenium.webdriver.common.keys import Keys

# id 입력
driver \
    .find_element_by_xpath('//*[@id="id"]') \
    .send_keys('my_id')

# password 입력 + 엔터
driver \
    .find_element_by_xpath('//*[@id="pw"]') \
    .send_keys('my_password' + Keys.ENTER)
send_keys: 해당 요소에 나열된 key sequence를 순차적으로 입력
Keys: 특수 키를 쓸 수 있게 해주는 class로 ENTER, CONTROL, SHIFT 등 모든 특수키가 있으며 항상 대문자로 작성
Point II
마우스 입력

# 로그인 버튼 클릭
driver \
    .find_element_by_xpath('//*[@id="log.login"]') \
    .click()
click: 해당 요소를 클릭하는 효과


4. 다양한 입력, Action Chains
Point I

기본 입력의 한계: 각 요소에 key sequence를 입력하거나 마우스 클릭 등의 액션을 할 수 있지만, 더 이상의 복잡한 액션을 할 수 없습니다.
Action Chains: 여러 가지의 action을 chain처럼 엮어 수행하는 기능으로 이를 통해 좀 더 복잡한 액션을 할 수 있습니다.
Point II


Point III
ActionChains의 키보드 입력

# 요소 찾기
_id = driver.find_element_by_xpath('//*[@id="id"]')
_pw = driver.find_element_by_xpath('//*[@id="pw"]')

# 액션 수행
webdriver.ActionChains(driver) \
    .send_keys_to_element(_id, 'my_id') \
    .send_keys_to_element(_pw, 'my_pw') \
    .send_keys(Keys.ENTER) \
    .perform()
send_keys_to_element: 주어진 요소에 주어진 key sequence를 입력하여 하나의 ActionChains 내에서 여러 요소에 액션을 수행
send_keys: 요소가 주어지지 않기 때문에 현재 커서가 위치한 곳에서 key sequence를 입력
Point IV
ActionChains의 다양한 method 조합

# 컨트롤 + click
_button = driver.find_element_by_xpath('//button')
chains = webdriver.ActionChains(driver)

chains.key_down(Keys.CONTROL) # 컨트롤 버튼 누른 채로
chains.click(_button) # 버튼 요소 클릭
chains.perform()

# 마우스 커서를 올렸다, 3초 뒤에 떼기
_button = driver.find_element_by_xpath('//button')
chains = webdriver.ActionChains(driver)

chains.move_to_element(_button) \
    .pause(3) \
    .move_by_offset(100, 100) \
    .perform()
move_to_element: 주어진 요소로 마우스 커서를 이동
pause: n초만큼 멈췄다 뒤의 액션을 수행
move_by_offset: x, y축으로 주어진 수치만큼 이동
